In [1]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, roc_auc_score, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Conv2D, BatchNormalization, Dropout, MaxPooling1D, MaxPooling2D, Flatten, Reshape

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
start_date = '2011-01-01'
end_date = '2020-12-31'
dataset_folder = './dataset/open-meteo'
# dataset docs https://open-meteo.com/en/docs

## Import data

## Feature Engineering

In [6]:
mock = np.random.rand(13, 2)
mock[:, 0] = mock[:, 0] * 1e+7
mock[:, 1] = np.round(mock[:, 1], 3)

In [7]:
mock = mock[np.newaxis, np.newaxis, :, :]

## QNet

In [27]:
model = Sequential()

model.add(keras.layers.Input(shape=(1, 13, 2)))

model.add(Conv2D(filters=2, kernel_size=(1,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=2, kernel_size=(1,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.2))

model.add(Dense(1, activation='tanh'))

opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='mean_squared_errors')

model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 1, 13, 2)       │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 1, 13, 2)       │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 1, 13, 2)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 1, 13, 2)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 1, 13, 2)       │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 1, 13, 2)       │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 1, 13, 2)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 1, 13, 2)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1, 13, 1)       │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31 (124.00 B)

 Trainable params: 23 (92.00 B)

 Non-trainable params: 8 (32.00 B)

In [29]:
mock

array([[[[3.74540119e+06, 9.51000000e-01],
         [7.31993942e+06, 5.99000000e-01],
         [1.56018640e+06, 1.56000000e-01],
         [5.80836122e+05, 8.66000000e-01],
         [6.01115012e+06, 7.08000000e-01],
         [2.05844943e+05, 9.70000000e-01],
         [8.32442641e+06, 2.12000000e-01],
         [1.81824967e+06, 1.83000000e-01],
         [3.04242243e+06, 5.25000000e-01],
         [4.31945019e+06, 2.91000000e-01],
         [6.11852895e+06, 1.39000000e-01],
         [2.92144649e+06, 3.66000000e-01],
         [4.56069984e+06, 7.85000000e-01]]]])

In [28]:
model.predict(mock)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


array([[[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]]], dtype=float32)